In [4]:
import numpy as np
import pandas as pd
import yfinance as yf
import networkx as nx
from itertools import combinations
from threading import Thread

In [5]:
start_date = '2022-01-01'
end_date = '2022-11-01'

investment = 500000

max_weighting = 0.25
min_weighting = 1 / (12 * 2)

In [7]:
tickers_raw = pd.read_csv('Tickers.csv', header=None)[0].tolist()

tickers = []
stocks_info = {}
stocks_hist = {}

def validate_stock(ticker):
    
    stock_info = yf.Ticker(ticker).info
    
    try:
        if stock_info['quoteType'] != 'EQUITY':
            raise Exception
        
        if stock_info['currency'] == 'USD' and stock_info['market'] == 'us_market':
            
            stock_hist = yf.Ticker(ticker).history(start=start_date, end=end_date, interval='1d').dropna()
            
            stock_monthly_trading_days = stock_hist['Volume'].groupby(pd.Grouper(freq='MS')).count()
            stock_monthly_volume = stock_hist['Volume'].groupby(pd.Grouper(freq='MS')).sum()
            
            for month in stock_monthly_trading_days.index:
                if stock_monthly_trading_days.loc[month] < 20:
                    stock_monthly_volume.drop(month, inplace=True)
            
            if stock_monthly_volume.mean() >= 200000:
                tickers.append(ticker)
                stocks_info[ticker] = stock_info
                stocks_hist[ticker] = stock_hist
            else:
                print(f'{ticker}: Ticker does not meet average monthly volume requirements')
        else:
            print(f'{ticker}: Ticker does not reference a stock denominated in USD')
    except:
        print(f'{ticker}: Ticker does not reference a valid stock')

threads = []

for ticker in tickers_raw:
    thread = Thread(target=validate_stock, args=[ticker])
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

- AGN: No summary info found, symbol may be delisted
AGN: Ticker does not reference a valid stock
- CELG: No summary info found, symbol may be delisted
CELG: Ticker does not reference a valid stock


Exception in thread Thread-107 (validate_stock):
Traceback (most recent call last):
  File "c:\Users\ansme\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\ansme\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ansme\AppData\Local\Temp\ipykernel_3580\1525425717.py", line 9, in validate_stock
  File "c:\Users\ansme\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\ticker.py", line 138, in info
    return self.get_info()
           ^^^^^^^^^^^^^^^
  File "c:\Users\ansme\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\base.py", line 894, in get_info
    data = self._quote.info
           ^^^^^^^^^^^^^^^^
  File "c:\Users\ansme\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\scrapers\quote.py", line 27, in info
    self._scrape(self.proxy)
  File "c:\Users\ansme\AppData\Local

TWX: Ticker does not reference a valid stock


Exception in thread Thread-120 (validate_stock):
Traceback (most recent call last):
  File "c:\Users\ansme\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\ansme\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ansme\AppData\Local\Temp\ipykernel_3580\1525425717.py", line 9, in validate_stock
  File "c:\Users\ansme\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\ticker.py", line 138, in info
    return self.get_info()
           ^^^^^^^^^^^^^^^
  File "c:\Users\ansme\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\base.py", line 894, in get_info
    data = self._quote.info
           ^^^^^^^^^^^^^^^^
  File "c:\Users\ansme\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\scrapers\quote.py", line 27, in info
    self._scrape(self.proxy)
  File "c:\Users\ansme\AppData\Local

- MON: No summary info found, symbol may be delisted
MON: Ticker does not reference a valid stock
RY.TO: Ticker does not reference a stock denominated in USD
TD.TO: Ticker does not reference a stock denominated in USD
- RTN: No summary info found, symbol may be delisted
RTN: Ticker does not reference a valid stock
- PCLN: No summary info found, symbol may be delisted
PCLN: Ticker does not reference a valid stock


In [8]:
beta_dict = {}

market_return = yf.Ticker('^GSPC').history(start=start_date, end=end_date, interval='1d').dropna()['Close'].pct_change()
market_variance = market_return.var()

for ticker in tickers:
    stock_return = stocks_hist[ticker]['Close'].pct_change()
    covariance = pd.DataFrame({ticker: stock_return, 'S&P 500': market_return}).cov().loc[ticker, 'S&P 500']
    beta_dict[ticker] = covariance / market_variance

beta_df = pd.DataFrame(beta_dict.items(), columns=['Ticker', 'Beta'])
beta_df.sort_values(by=['Beta'], ascending=False, ignore_index=True, inplace=True)

In [9]:
corr_dict = {}

for ticker_1 in tickers:
    stock_return_1 = stocks_hist[ticker_1]['Close'].pct_change()
    for ticker_2 in tickers:
        if ticker_1 != ticker_2:
            stock_return_2 = stocks_hist[ticker_2]['Close'].pct_change()
            corr = pd.DataFrame({ticker_1: stock_return_1, ticker_2: stock_return_2}).corr().loc[ticker_1, ticker_2]
            corr_dict[f'{ticker_1} {ticker_2}'] = corr

In [10]:
edge_dict = {}

for ticker_1 in tickers:
    for ticker_2 in tickers:
        if ticker_1 != ticker_2:
            weight = beta_dict[ticker_1] * beta_dict[ticker_2] * corr_dict[f'{ticker_1} {ticker_2}'] ** 2
            edge_dict[f'{ticker_1} {ticker_2}'] = (ticker_1, ticker_2, weight)

In [11]:
tickers_top = beta_df['Ticker'].iloc[:20].to_list()

tickers_top_combinations = combinations(tickers_top, 12)

In [12]:
tc_mst_size_list = []

def tc_mst_size(tc):
    tc_edge_list = []
    
    for index_1 in range(12):
        for index_2 in range(index_1, 12):
            if index_1 != index_2:
                tc_edge_list.append(edge_dict[f'{tc[index_1]} {tc[index_2]}'])
    
    tc_graph = nx.Graph()
    
    tc_graph.add_weighted_edges_from(tc_edge_list)
    tc_graph_mst = nx.maximum_spanning_tree(tc_graph)
    
    tc_mst_size_list.append((tc, tc_graph_mst.size(weight='weight')))

threads = []

for tc in tickers_top_combinations:
    thread = Thread(target=tc_mst_size, args=[tc])
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [13]:
tc_optimal = max(tc_mst_size_list, key=lambda tup : tup[1])[0]
print(f'Optimal combination of stocks: {", ".join(tc_optimal)}')

Optimal combination of stocks: PYPL, AMZN, QCOM, GM, GOOG, BLK, AAPL, COF, MSFT, ACN, AXP, TXN


In [14]:
options_ratios = []
options_tickers = []

def get_options_data(ticker):
    try:
        options_data = yf.Ticker(ticker).option_chain(yf.Ticker(ticker).options[0])
        
        puts_sum = options_data.puts['volume'].sum()
        calls_sum = options_data.calls['volume'].sum()
        
        puts_to_calls = puts_sum / calls_sum
        calls_to_puts = calls_sum / puts_sum
        
        options_ratios.append({'Ticker': ticker, 'pcr': puts_to_calls, 'cpr': calls_to_puts})
        options_tickers.append(ticker)
    except:
        print(f'{ticker}: No options data available')

threads = []

for ticker in tc_optimal:
    thread = Thread(target=get_options_data, args=[ticker])
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [15]:
for ticker in tc_optimal:
    if ticker not in options_tickers:
        options_ratios.append({'Ticker': ticker, 'pcr': 1, 'cpr': 1})

options_df = pd.DataFrame(options_ratios, columns=['Ticker', 'pcr', 'cpr'])

In [16]:
weight_df = pd.DataFrame()

pcr_avg = options_df.pcr.mean() + options_df.pcr.median()
cpr_avg = options_df.cpr.mean() + options_df.cpr.median()

weight_df[['Ticker', 'weight']] = options_df[['Ticker', 'pcr']] if pcr_avg > cpr_avg else options_df[['Ticker', 'cpr']]

weight_df.sort_values(by=[weight_df.columns[1]], ascending=False, ignore_index=True, inplace=True)

weight_max = weight_df.weight.max()
weight_min = weight_df.weight.min()

if weight_max == weight_min:
    weight_df.weight = weight_df.weight / weight_df.weight.sum()
else:
    weight_df.weight = (weight_df.weight - weight_min) / (weight_df.weight.sum() - weight_min * 12)
    
    weight_df.weight.where(weight_df.weight < max_weighting, max_weighting, inplace=True)
    weight_df.weight.where(weight_df.weight > min_weighting, min_weighting, inplace=True)
    
    delta = 1 - weight_df.weight.sum()
    
    for i in range(len(weight_df.index)):
        if delta > 0:
            delta_i = min(delta, max_weighting - weight_df.loc[i, 'weight'])
            weight_df.loc[i, 'weight'] += delta_i
        elif delta < 0:
            delta_i = max(delta, min_weighting - weight_df.loc[11 - i, 'weight'])
            weight_df.loc[11 - i, 'weight'] += delta_i
        else:
            continue
        delta -= delta_i

weight_df

,Ticker,weight
0,COF,0.250000
1,PYPL,0.203229
2,TXN,0.093695
3,MSFT,0.070122
4,GOOG,0.063070
5,AMZN,0.059515
6,AAPL,0.048852
7,QCOM,0.044849
8,ACN,0.041667
9,GM,0.041667


Save the ticker, price, shares invested, total value (price × shares), and portfolio weight of each stock into a DataFrame.

In [17]:
Portfolio_Final = pd.DataFrame(columns=['Ticker', 'Price', 'Shares', 'Value', 'Weight'])

Portfolio_Final.Ticker = weight_df.Ticker
Portfolio_Final.Weight = weight_df.weight
Portfolio_Final.Price = Portfolio_Final.Ticker.apply(lambda ticker : stocks_info[ticker]['previousClose'])
Portfolio_Final.Value = Portfolio_Final.Weight * investment
Portfolio_Final.Shares = Portfolio_Final.Value / Portfolio_Final.Price

Portfolio_Final.index = range(1, len(Portfolio_Final.index) + 1)

Portfolio_Final

,Ticker,Price,Shares,Value,Weight
1,COF,101.79,1228.018469,125000.000000,0.250000
2,PYPL,78.54,1293.790948,101614.341093,0.203229
3,TXN,178.87,261.909172,46847.693515,0.093695
4,MSFT,235.77,148.708102,35060.909293,0.070122
5,GOOG,92.26,341.807896,31535.196443,0.063070
6,AMZN,95.09,312.942890,29757.739439,0.059515
7,AAPL,133.49,182.980370,24426.049532,0.048852
8,QCOM,118.39,189.414117,22424.737352,0.044849
9,ACN,281.08,74.118875,20833.333333,0.041667
10,GM,37.82,550.854927,20833.333333,0.041667


Ensure that the portfolio total adds up to $\$500000$, and that the weights add to $100\%$.

In [18]:
print(f'Total value: ${round(Portfolio_Final.Value.sum(), 2):.2f}')
print(f'Total weight: {round(Portfolio_Final.Weight.sum() * 100, 2):.2f}%')

Total value: $500000.00
Total weight: 100.00%


Write the `Ticker` and `Shares` data from `Portfolio_Final` to a csv file.

In [19]:
Stocks_Final = Portfolio_Final[['Ticker', 'Shares']]
Stocks_Final.to_csv('Stocks_Group_01.csv')